In [13]:
# IMPORTS
import tensorflow as tf
import numpy as np
import random
from PIL import Image
from tensorflow.python.keras.models import Input, Model
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, concatenate, Dense, Conv3D, MaxPool3D
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img 

ImportError: cannot import name '_imaging' from 'PIL' (C:\Users\labuser\AppData\Roaming\Python\Python37\site-packages\PIL\__init__.py)

In [3]:
# CONSTANTS
BSE_SHAPE = (500, 500, 1)
EBSD_SHAPE = (200, 200, 3)
OM_SHAPE = (1000, 1000, 1)


In [4]:
# VARIABLES
CONV_NODES = [64, 128, 256, 512]
DENSE_NODES = [32, 64, 128]

NUM_CONV = [0, 1]
NUM_DENSE = [0, 1, 2]


In [5]:
# TRAIN DATA IMPORT
BSE = 'path'
EBSD = 'path'
OM = 'path'


In [6]:
# # #### testing network ####
# test_path = 'test_data/'
# 
# # load test img
# i = load_img('test_data/0/bse.png')
# x = img_to_array(i)


In [7]:
# #### Build Model ####
# INPUTS
input_1 = Input(shape=(100,100,1))
input_2 = Input(shape=(100,100,3))
input_3 = Input(shape=(100,100,1))

# MODEL

# FIRST BRANCH
# BSE
# needs to accept a black and white image
one = Conv2D(64, (3, 3), activation="relu")(input_1)
one = MaxPool2D(pool_size=(2, 2))(one)

one = Conv2D(64, 3, activation='relu')(one)
one = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(one)

one = Model(inputs=input_1, outputs=one)

# SECOND BRANCH
# EBSD
# needs to accept a RGB image
two = Conv2D(64, (3, 3), activation='relu')(input_2)
two = MaxPool2D(pool_size=(2, 2))(two)

two = Conv2D(64, 3, activation='relu')(two)
two = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(two)

two = Model(inputs=input_2, outputs=two)

# THIRD BRANCH
# OM
# needs to accept a black and white image
three = Conv2D(64, (3, 3), activation='relu')(input_3)
three = MaxPool2D(pool_size=(2, 2))(three)

three = Conv2D(64, 3, activation='relu')(three)
three = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(three)

three = Model(inputs=input_3, outputs=three)

# COMBINE THE BRANCHES
# concatenates the three branches all together into one
combined = concatenate([one.output, two.output])
combined = concatenate([combined, three.output])

# END BRANCH
end = Dense(64, activation='relu')(combined)
end = Dense(1, activation='sigmoid')(end)

# MODEL
model = Model(inputs=[one.input, two.input, three.input], outputs=end)

# COMPILE MODEL
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])



In [14]:
# IMPORT DATA

# load in data as an array of arrays which contain [bse, ebsd, om, stress]
train_data = []
for i in range(10):
    path = 'test_data/SR/train/' + str(i) + '/'
    bse = np.array(load_img(path + 'bse.png'))
    bse = bse.resize((100, 100, 1))
    bse = tf.keras.utils.normalize(bse, axis=1)
    
    ebsd = np.array(load_img(path + 'ebsd.png'))
    ebsd = ebsd.resize((100, 100, 3))
    ebsd = tf.keras.utils.normalize(ebsd, axis=1)
    
    om = np.array(load_img(path + 'om.png'))
    om = om.resize((100, 100, 1))
    om = tf.keras.utils.normalize(om, axis=1)
    
    train_data.append([bse, ebsd, om, 1060])
    
for i in range(10):
    path = 'test_data/HT/train/' + str(i) + '/'
    bse = np.array(load_img(path + 'bse.png'))
    bse = bse.resize((100, 100, 1))
    bse = tf.keras.utils.normalize(bse, axis=1)
    
    ebsd = np.array(load_img(path + 'ebsd.png'))
    ebsd = ebsd.resize((100, 100, 3))
    ebsd = tf.keras.utils.normalize(ebsd, axis=1)
    
    om = np.array(load_img(path + 'om.png'))
    om = om.resize((100, 100, 1))
    om = tf.keras.utils.normalize(om, axis=1)
    
    train_data.append([bse, ebsd, om, 918])

# shuffle the data array

random.shuffle(train_data)

# load data into input data
bse_train = []
ebsd_train = []
om_train = []
stress_train = []

for i in range(len(train_data)):
    bse_train.append(train_data[i][0])
    ebsd_train.append(train_data[i][1])
    om_train.append(train_data[i][2])
    stress_train.append(train_data[i][3])


# reshape the arrays of each thing
bse_train.reshape(20, 100, 100, 1)
ebsd_train.reshape(20, 100, 100, 3)
om_train.reshape(20, 100, 100, 1)
stress_train.reshape(20, 1)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [0]:
# FIT MODEL
# callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,
                                                      write_graph=True, write_images=True)

model.fit([bse_train, ebsd_train, om_train],
          [stress_train], 
          epochs = 1,
          validation_split=0.2, 
          callbacks=[tensorboard_callback])
